In [4]:
import numpy as np
from cluster_sim.app.holes import Holes
from cluster_sim.app.grid import Grid
from cluster_sim.app.utils import get_node_coords

import os
from joblib import delayed
import time
from analysis_funcs import algorithm1, path_percolation, apply_error_channel, ParallelTqdm

os.makedirs("./data", exist_ok=True)

shape = [10, 10, 100]
seed = 12345

samples = 1

# The probabilities to simulate
p_vec = np.linspace(0, 0.3, 10)

# Input vector for all jobs
jobs_input_vec = p_vec

In [30]:
def main(input_params):
    """
    Main function for parallel processing. Here, we:

    1. Create a grid of the given shape
    2. Apply the error channel to the grid
    3. Generate a RHG lattice
    4. Look at clusters of the RHG lattice
    5. Return the results

    Returns:
        - p: The input parameter for the simulation
        - percolates: The number of times percolation occurred (True: 1, False: 0)
        - times: The average time taken for the simulation
    """
    
    p = input_params

    # Sanity check: check that this is equal to the move_list on the app
    # print(np.reshape(np.argwhere(removed_nodes == True), shape=-1))
    
    percolates = 0
    
    times = np.zeros(samples)
    for i in range(samples):
        start = time.time()
        G = Grid(shape)
        removed_nodes = np.zeros(shape[0] * shape[1] * shape[2], dtype=bool)

        G, D, removed_nodes = apply_error_channel(p, seed + i, shape, removed_nodes, G)

        # Generate an RHG lattice out of G
        G, removed_nodes, offsets = algorithm1(G, D, removed_nodes, shape)

        xoffset, yoffset, zoffset = offsets

        cubes, _ = D.find_lattice(removed_nodes, xoffset, yoffset, zoffset)

        C = D.build_centers_graph(cubes)
        connected_cubes = D.find_connected_lattice(C)

        connected_cubes.sort(key=lambda x: len(x.nodes), reverse=True)

        for c in connected_cubes:
            print(c)

        #X = D.connected_cube_to_nodes(connected_cubes[0])

        end = time.time() - start 

        times[i] = end

    return (p, percolates, times.mean())


In [31]:
main(0.05)

Graph with 634 nodes and 1838 edges
Graph with 1 nodes and 0 edges
Graph with 1 nodes and 0 edges
Graph with 1 nodes and 0 edges
Graph with 1 nodes and 0 edges


(0.05, 0, np.float64(0.2089390754699707))